In [ ]:
# import
import numpy as np
import optuna
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import xgboost as xgb

In [ ]:
with open('preprocessed_data/preprocessed_LSTM_CNN.pkl', 'rb') as f:
    LSTM_CNN_preprocessed_data = pickle.load(f)

In [ ]:
LSTM_CNN_preprocessed_data[4][8][2]

In [ ]:
# The variable LSTM_CNN_preprocessed_data is a list of our 5 bins [0-4][][]. Each bin contains the 15 belonging stocks or ETFs [0-4][0-14][].
# After accessing the stock we can get the we are left with a 3 element list containing [0] X the features, [1] y the target and [2] the ticker symbol [0-4][0-14][0-2].

# For the NNs the accessed data is 3D (samples, sample_size, features_per_sample)

LSTM_CNN_preprocessed_data[2][7][0].shape
# This for example accesses the X data from the 8th stock in bin 3.

In [ ]:
# we pull out the model creation process for clarity

def build_cnn_model(params, X_train):
    # create the model
    model = Sequential()
    for k in range(params['n_layers']):
        if (k == 0): 
            model.add(Conv1D(filters=params['n_filters'], padding="same",kernel_size=params['kernel_size'], activation="relu",
                        input_shape=(X_train.shape[1], X_train.shape[2])))
        else:
            model.add(Conv1D(filters=params['n_filters'], padding="same",kernel_size=params['kernel_size'], activation="relu"))
        model.add(MaxPooling1D(pool_size=params['pool_size']))
        model.add(Dropout(rate=params['dropout_rate']))
    model.add(Flatten())
    model.add(Dense(units=params['dense_units'], activation="relu"))
    model.add(Dense(1, activation="linear"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss="mse")

    return model

In [ ]:
# Only pass pandas DataFrames as data_X data_y to the function below.
# create windows 
def create_windows(data_X, data_y, window):
    X, y, indices = [], [], []
    for i in range(window, len(data_X)):
        X.append(data_X[i - window:i].values.astype("float32"))
        y.append(float(data_y.iloc[i-1])) # i-1 because we shifted the target by one in the General preprocessing pipeline 
        indices.append(data_y.index[i-1])
    return np.array(X, dtype="float32"), np.array(y, dtype="float32"), np.array(indices)

#Create our CNN time series objective with data sets
def creat_cnn_objective(X, y):
    def cnn_objective(trial):
        # Suggest hyperparameters
        n_filters = trial.suggest_int("n_filters", 16, 128, step=16)
        n_layers = trial.suggest_int("n_layers", 1, 2)
        kernel_size = trial.suggest_int("kernel_size", 2, 8)
        pool_size = trial.suggest_int("pool_size", 2, 4)
        dense_units = trial.suggest_int("dense_units", 16, 128, step=16)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5, step=0.1)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
        min_window_size = ((pool_size + (kernel_size-1)) * pool_size) + (kernel_size-1) + 10 #calculate min window size based on other hyperparameters and add a margin of 10
        window_size = trial.suggest_int("window_size", min_window_size, 110, step=10)


        # Start create windows and split the data
        # training 0.9, validation 0.1, test 0.1
        X_windows, y_windows, indices = create_windows(X, y, window_size)

        end_train_set = X.index[int(X.shape[0] * 0.8)]
        end_validation_set = X.index[int(X.shape[0] * 0.9)]

        train_mask = indices < end_train_set
        validation_mask = (indices >= end_train_set) & (indices < end_validation_set)
        test_mask = indices >= end_validation_set

        X_train = X_windows[train_mask]
        y_train = y_windows[train_mask]
        X_val = X_windows[validation_mask]
        y_val = y_windows[validation_mask]
        X_test = X_windows[test_mask]
        y_test = y_windows[test_mask]
        # End create windows

        # parameters for the model to pass further to the build_cnn_model function
        params = {
            "n_filters": n_filters,
            "n_layers": n_layers,
            "kernel_size": kernel_size,
            "pool_size": pool_size,
            "dense_units": dense_units,
            "dropout_rate": dropout_rate,
            "learning_rate": learning_rate
        }

        # create the model
        model = build_cnn_model(params, X_train)

        early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

        # Train
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=batch_size,
            callbacks=[early_stop],
            verbose=0
        )

        # compute errors
        y_pred = model.predict(X_val, batch_size=batch_size)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mape = np.mean(np.abs((y_pred - y_val) / y_val)) * 100
        nrmse = np.sqrt(mean_squared_error(y_val, y_pred)) / np.mean(y_val) * 100  # in percent

        # store the results in the trial
        trial.set_user_attr("mae", mae)
        trial.set_user_attr("r2", r2)
        trial.set_user_attr("rmse", rmse)
        trial.set_user_attr("nrmse", nrmse)
        
        return mape
    return cnn_objective


In [ ]:
# number of trials per asset
N_trials = 70

# outer list to store results for each bin
CNN_data_results = []

for i in range(len(LSTM_CNN_preprocessed_data)):
    
    # inner list to store results for each asset
    CNN_bin_results = []
    for j in range(len(LSTM_CNN_preprocessed_data[i])):
        X, y, symbol = LSTM_CNN_preprocessed_data[i][j]


        # 5.2. CNN study
        study = optuna.create_study(
            direction="minimize",
            study_name="cnn_regression_study",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        )
        study.optimize(creat_cnn_objective(X, y), n_trials=N_trials)

        params = study.best_params

        # create windows and split the data
        # training 0.8, validation 0.1, test 0.1
        X_windows, y_windows, indices = create_windows(X, y, params['window_size'])

        end_train_set = X.index[int(X.shape[0] * 0.8)]
        end_validation_set = X.index[int(X.shape[0] * 0.9)]

        train_mask = indices < end_train_set
        validation_mask = (indices >= end_train_set) & (indices < end_validation_set)
        test_mask = indices >= end_validation_set

        X_train = X_windows[train_mask]
        y_train = y_windows[train_mask]
        X_val = X_windows[validation_mask]
        y_val = y_windows[validation_mask]
        X_test = X_windows[test_mask]
        y_test = y_windows[test_mask]

        # create the model
        model = build_cnn_model(params, X_train)

        # Train
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=params['batch_size'],
            verbose=0
        )

        # compute errors
        y_pred = model.predict(X_test, batch_size=params['batch_size'])
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_mae = mean_absolute_error(y_test, y_pred)
        test_r2 = r2_score(y_test, y_pred)
        test_mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100
        test_nrmse = np.sqrt(mean_squared_error(y_test, y_pred)) / np.mean(y_test) * 100  # in percent

        # store the results in the trial
        results = {
            "test_mape": test_mape,
            "test_rmse": test_rmse,
            "test_mae":  test_mae,
            "test_r2":   test_r2
        }


        CNN_bin_results.append(results)
    # write out the results for each bin
    with open(f'results_ret_log/CNN/Performance_metrices_CNN_bin_{i}.pkl', 'wb') as f:
        pickle.dump(CNN_bin_results, f)
    CNN_data_results.append(CNN_bin_results)
# write out full results
with open(f'results_ret_log/CNN/Performance_metrices_CNN_full.pkl', 'wb') as f:
    pickle.dump(CNN_data_results, f)